<a href="https://colab.research.google.com/github/MJMortensonWarwick/Programming_and_Big_Data_Analytics_2425/blob/main/2_03_Calculated_Fields%2C_Indexing_and_Subsetting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1vv_PsWBnUJwSCkwKDoJAC-vXjtaEA4Ts)

# 2.03 Pandas: Calculated fields, indexing and subsetting dataframes
### Calculated Fields
In many forms of data analysis often we will want to generate new fields/columns based on existing data or as some combination of them. In other words, this is data that we are calculating rather than reading.

We will create some calculated fields based on the dictionary input from the last session. If you have started a new notebook run the below code to recreate the dataframe:

In [ ]:
import pandas as pd
import numpy as np

# create a dictionary of orders
orders = {'o10001':{'date':'2025/01/10', 'product':'Blockchain database', 'quantity':'1'},
            'o10002':{'date':'2025/01/13', 'product':'Stock market prediction engine', 'quantity':'2'},
            'o10003':{'date':'2025/01/14', 'product':'Portfolio optimisation tool', 'quantity':'10'},
            'o10004':{'date':'2025/01/15', 'product':'Man\'s suit', 'quantity':'2'}
}

# convert to a dataframe
orders_df = pd.DataFrame(orders)

# create a dicitonary of products
products = {'123':{'name':'Blockchain database', 'cost_price':12.12, 'sale_price':15.00},
            '124':{'name':'Stock market prediction engine', 'cost_price':2.15, 'sale_price':9.99},
            '125':{'name':'Portfolio optimisation tool', 'cost_price':22.45, 'sale_price':49.99},
            '126':{'name':'Financial services chatbot', 'cost_price':0.45, 'sale_price':2.99},
            '127':{'name':'Man\'s suit', 'cost_price':0.78, 'sale_price':1.49}
}

# convert to a dataframe
products_df = pd.DataFrame(products)

# transpose (flip on their axes) both dataframes
orders_df = orders_df.transpose()
products_df = products_df.transpose()

# join (left) orders_df and products_df
joined_df = orders_df.merge(products_df, how='left', left_on='product', right_on='name')

# drop the repeated column and display on screen
joined_df = joined_df.drop(['name'], axis=1)
joined_df

Expanding our example, we now want to know the total cost price and total sale price for each order, calculated by multiplying the unity prices by the quantity. Again, pandas makes this relatively easy to do … in theory at least. The following code would calculate, working much like a normal calculation on a single variable (as opposed to a whole column):

In [ ]:
joined_df['total_cost_price'] = joined_df['quantity'] * joined_df['cost_price']
joined_df['total_sale_price'] = joined_df['quantity'] * joined_df['sale_price']
joined_df

Uh-oh ... error. More specifically the message "TypeError: can't multiply sequence by non-int of type 'float'". Although the description is a little technical, we can get the gist of the problem - there is some form of "type" error preventing the multiplication. Let's therefore evaluate what data types we have in our dataframe using a print statement:

In [ ]:
print(joined_df.dtypes)

We would want our quantity, cost_price and sale_price to be of a numeric type (float or integer). In this case everything is listed as an "object". Object is pandas' most flexible data type (dtype) designed to work with "text or mixed numeric and non-numeric values". While pandas will try to infer the relevant type for data, the fallback option is to assign as object which is what has happened here. However, we can fix this fairly easily with a for loop that converts the relevant fields to floats. An alternative approach, particularly when you have many columns of data, is to ask pandas to convert them as a set using convert_dtypes (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.convert_dtypes.html).

In [ ]:
field_list = ['quantity', 'cost_price', 'sale_price']
for field in field_list:
    joined_df[field] = joined_df[field].astype(float)

To explain what we are doing here.
1. First, we have created a list of the fields (columns) we want to change ('quantity', 'cost_price' and 'sale_price') called field_list.
2. Secondly we have created a __for__ loop that iterates through this list.
3. For each of the records in _joined\_df_ we index (you\'ll see this below) the dataframe by that particular field. I.e. we'll find that particular field.
4. Lastly, we change the __type__ of that dataitem from "object" (i.e. a __string__) to "float" (i.e. a decimal number).

With this in place we can run the earlier code without error:

In [ ]:
joined_df['total_cost_price'] = joined_df['quantity'] * joined_df['cost_price']
joined_df['total_sale_price'] = joined_df['quantity'] * joined_df['sale_price']
joined_df

### Indexing and Subsetting Dataframes
As with strings and lists, we can index/slice pandas dataframes based on a range of criteria. Note, this is an area where the recommended pandas syntax has changed significantly over the years. In the past .ix (index) was the preferred syntax, and you will still often see this in older libraries/tutorials. Today, however, _.loc_ (location) and _.iloc_ (index location) are the prefered options and the ones we will use here. We will start by indexing item zero (the first item) in our dataframe:

In [ ]:
joined_df.iloc[0]

Although the command is slightly different than when working with strings and lists, the principles remain the same, and it is also possible to use slices here by including a colon:

In [ ]:
joined_df.iloc[0:2]

We can view specific columns by calling them by name. Note, we used this feature in our earlier for loop to change dtypes.

In [ ]:
joined_df['product']

The same techniques can also be used to create subsets of the data:

In [ ]:
df_subset_one = joined_df.iloc[0:2]
df_subset_one

In [ ]:
df_subset_two = joined_df[['product', 'quantity', 'total_sale_price']]
df_subset_two

Note that subsetting by rows follows the normal Python convention where the first number is the first item to return and the second is the value after the last item to return. Subsetting by columns requires a double set of square brackets (e.g. [['a', 'b', 'c']]).

### Exporting Dataframes
Finally, we can use Pandas to efficiently export our Dataframe to file. There are multiple export options available, see the documentation for more details: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#conversion, to include Excel, many major databases, and even HTML. However, typically the most useful is to CSV (for reusability):

In [ ]:
joined_df.to_csv('Joined_Dataframe.csv', sep=",")

Note the sep="," denotes that the columns will be sepearated by a comma - hence the file type CSV (Comma Separated Values). Click on the folder icon on the left to download and check your file.

### EXERCISES
1. Using _product\_df_, can you create a calculated field for the amount of markup associated with each product? The formula for this should be:
$markup = \frac{(sale \hspace{0.1cm} price \hspace{0.2cm} - \hspace{0.2cm} cost \hspace{0.2cm} price)}{cost \hspace{0.1cm} price}$

2. Can you also create a calculated field for the percentage markup?

3. Working this time with _joined\_df_, can you create a subset of the dataframe where the total sale price is less than £20.00?